In [1]:
from Model import *
from DataLoader import *
from Setting import *
import collections
import os
import numpy as np
import pickle
import heapq

## Week3 code1 Median Maintenance with heap

In [14]:
# heap class, using heapq package
class Heap(object):
    def __init__(self, initial=None, key=lambda x:x , reverse = False):
        self.key = key
        if initial:
            self._data = [(key(item),id(item), item) for item in initial]
            heapq.heapify(self._data)
        else:
            self._data = []

    def push(self, item):
        heapq.heappush(self._data, (self.key(item),id(item), item))

    def pop(self):
        try:
            return heapq.heappop(self._data)[2]
        except:
            return None

    def length(self):
        return len(self._data)

In [25]:
# Heap class implementation using list
class Heap(object):
    
    def __init__(self):
        self._data = [4,4,8,9,4,12,9,11,13]
        
    def insert(self, item):
        self._data.append(item)
        location = len(self._data)
        
        while item <= self._data[location//2 -1]:
            self._data[location-1], self._data[location//2 -1] = self._data[location//2 -1],self._data[location-1] 
            location = location//2
        print(self._data)
    
    def popmin(self):
        minvalue = self._data[0]
        
        self._data[0], self._data[len(self._data)-1] = self._data[len(self._data)-1], self._data[0]
        self._data = self._data[:-1]
        
        
        data = self._data
        
        
        loc = 0
        nextloc1 = (loc+1)*2 -1
        nextloc2 = (loc+1)*2 
        
        
        
        while data[loc]>
                
            
        
        print(data)
        return minvalue
        

heap = Heap()

heap.insert(7)
heap.insert(10)
heap.insert(5)

assert heap.popmin() == 4

[4, 4, 8, 9, 4, 12, 9, 11, 13, 7]
[4, 4, 8, 9, 4, 12, 9, 11, 13, 7, 10]
[4, 4, 5, 9, 4, 8, 9, 11, 13, 7, 10, 12]
[12, 4, 5, 9, 4, 8, 9, 11, 13, 7, 10]


In [2]:
class TwoHeapsMedian(Model):
    def __init__(self):
        super().__init__()

        
    def preprocess(self):
        self.data = self.dataLoader.data
        self.codedict = {}
    
    

        
    def model(self):
        pass
        


In [46]:
processLine = lambda x : list(map(int,map(str.strip,x.split())))
arg = {'fileName':'./data/median.txt','numLines':None ,  'processLine' : processLine  }

d = DataLoader(**arg)
m = TwoHeapsMedian()
s = Setting(d,m,False)
s.run()

Loading data ...
Total 1001 lines read
--------------------------------------------------
Need bits: 48457275093
result:(19, 9)


In [ ]:
s